In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

import pandas as pd
import numpy as np

import datetime as dt

from flask import Flask, jsonify

In [2]:
# Database Setup
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)
Base.metadata.create_all(engine)

# Save reference to the tables.
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
# Flask Setup
app = Flask(__name__)

In [4]:
# List all routes that are available
@app.route("/")
@app.route("/")
def welcome():
    return (
        f"Welcome to the Surf's Up assignment API!<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation"
        f"/api/v1.0/stations"
        f"/api/v1.0/tobs"
        f"/api/v1.0/<start> and /api/v1.0/<start>/<end>"
    )

In [5]:
# Precipitation: `/api/v1.0/precipitation`

@app.route("/api/v1.0/precipitation")
def precipitation():
    session = Session(engine)
    sel = [Measurement.date, Measurement.prcp]
    query = session.query(*sel).all()
    session.close()

# Convert the query results to a dictionary using `date` as the key and `prcp` as the value

    precipitation = []
    for date, prcp in query:
        precipitation_dict = {}
        precipitation_dict["Date"] = date
        precipitation_dict["Precipitation"] = prcp
        precipitation.append(precipitation_dict)
    
# Return the JSON representation of your dictionary.

    return jsonify(precipitation)


In [6]:
# Stations: `/api/v1.0/stations`

@app.route("/api/v1.0/stations")
def stations():
    session = Session(engine)
    sel1 = [Station.id, Station.station, Station.name]
    query = session.query(*sel1).all()
    session.close()

# Convert the query results to a dictionary using `date` as the key and `prcp` as the value

    stations = []
    for id, station, name  in query:
        stations_dict = {}
        stations_dict["ID"] = id
        stations_dict["Station"] = station
        stations_dict["Name"] = name
        stations.append(stations_dict)
    
# Return the JSON representation of your dictionary

    return jsonify(stations)

In [7]:
# temperature observations `/api/v1.0/tobs`
#Query the dates and temperature observations of the most active station for the last year of data.


@app.route("/api/v1.0/stations")
def temps():
    session = Session(engine)
    #Find the latest date 
    order_by_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    latestdate = dt.datetime.strptime(order_by_date, '%Y-%m-%d')
    #Set dates for a year from the latest date 
    query_date = dt.date(latestdate.year -1, latestdate.month, latestdate.day)
    #find the most active station: USC00519281
#     most_active = session.query(Station.station).func_count(Station.station).desc().all()
#     most_active
    sel2 = [Measurement.date, Measurement.tobs]
    query = session.query(*sel2).filter(Measurement.date >= querydate).all()
    session.close()
    
    results = session.query(Measurement.date, Measurement.tobs).filter(Measurement.station == "USC00519281").\
                    filter(Measurement.date >= query_date).all()

    temps = []
    for date, tobs in query:
        temps_dict = {}
        temps_dict["Date"] = date
        temps_dict["Tobs"] = tobs
        temps.append(temps_dict)

# Return a JSON list of temperature observations (TOBS) for the previous year.
    return jsonify(temps)


In [8]:
# Start Date  `/api/v1.0/<start>` and `/api/v1.0/<start>/<end>`

@app.route("/api/v1.0/<start>")

def start(start):
    
    session = Session(engine)
    start_date = dt.datetime.strptime(start, '%Y-%m-%d')

# When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date.

    
    results = session.query(func.min(Measurement.tobs), 
                            func.max(Measurement.tobs), 
                            func.avg(Measurement.tobs)).\
                            filter(Measurement.date >= start_date).all()
    session.close()

    temp_list = []
    for min, max, avg in results:
        temp_dict = {}
        temp_dict["min"] = min
        temp_dict["max"] = max
        temp_dict["avg"] = avg
        temp_list.append(temp_dict)
        
#Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.
    return jsonify(temp_list)  

In [11]:
# End Date  `/api/v1.0/<start>` and `/api/v1.0/<start>/<end>`

@app.route("/api/v1.0/<start>")

def start_end(start, end):
    
    session    = Session(engine)
    start_date = dt.datetime.strptime(start, '%Y-%m-%d')
    end_date   = dt.datetime.strptime(start, '%Y-%m-%d')

# When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates greater than and equal to the start date.

    
    results = session.query(func.min(Measurement.tobs), 
                            func.max(Measurement.tobs), 
                            func.avg(Measurement.tobs)).\
                            filter(Measurement.date >= start_date).\
                            filter(Measurement.date >= end_date).all()
    session.close()

    temp_list = []
    for min, max, avg in results:
        temp_dict = {}
        temp_dict["min"] = min
        temp_dict["max"] = max
        temp_dict["avg"] = avg
        temp_list.append(temp_dict)
        
#Return a JSON list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.
    return jsonify(temp_list)  

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)